In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [5]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels = zip(*data)

## Inspect label counts

In [6]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [7]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels = zip(*balanced_data)

In [8]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


# Prepare 10 cross validation folds

In [9]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(train_index_set)),
    }


# Load features

In [10]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [11]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [12]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features['wav2vec_features-c'][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [13]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [14]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [15]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [17]:
class LangIdConvNet(nn.Module):
    def __init__(self):
        super(LangIdConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.5)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.5)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.5)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x
        

# Train Classification Models

In [18]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    
    return train_x, train_y, test_x, test_y

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y = get_data_for_fold(fold_id, feature_name, batch_size)
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [19]:
def train_on_fold(fold_id, feature_name, batch_size, epochs):
    torch.manual_seed(0)
    results = {}
    
    model = LangIdConvNet()

    train_loader, test_loader = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(LangIdConvNet(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs)
            )
            true_train_classes.extend(y)
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs)
            )

            true_test_classes.extend(y)

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc
        }
        
    return results
    

In [20]:
import csv
from pathlib import Path

def save_results(all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        Path('cnn_results_2').mkdir(exist_ok=True)
        fname = f"cnn_results_2/{feature_name}_{fold_index}.csv"
        with open(fname, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(
                ['epoch', 'train_loss', 'test_loss', 'train_acc', 'test_acc']
            )
            
            for epoch in sorted(result_entry['epochs'].keys()):
                writer.writerow([
                    epoch,
                    result_entry['epochs'][epoch]['train_loss'],
                    result_entry['epochs'][epoch]['test_loss'],
                    result_entry['epochs'][epoch]['train_acc'],
                    result_entry['epochs'][epoch]['test_acc']
                ])

In [21]:
all_results = []

for fold_index in cv_folds:
    for feature_name in raw_features:
        resutls = train_on_fold(fold_index, feature_name, batch_size=10, epochs=500)
        all_results.append({
            'fold_index': fold_index,
            'feature_name': feature_name,
            'epochs': resutls
        })
        save_results(all_results)

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2996]           4,611           4,611
         Dropout-2       [10, 3, 2996]               0               0
       MaxPool1d-3       [10, 3, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              30              30
         Dropout-5       [10, 3, 1496]               0               0
       MaxPool1d-6        [10, 3, 748]               0               0
          Conv1d-7        [10, 3, 746]              30              30
         Dropout-8        [10, 3, 746]               0               0
       MaxPool1d-9        [10, 3, 373]               0               0
         Conv1d-10        [10, 3, 371]              30              30
      MaxPool1d-11        [10, 3, 185]               0               0
Total params: 4,701
Trainable params: 4,701
Non-trainable params: 0
--------

Epoch: 200. Train Loss: 0.5678. Test Loss: 0.7046. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 210. Train Loss: 0.5493. Test Loss: 0.7069. Train Acc: 0.7059. Test Acc:0.6078
Epoch: 220. Train Loss: 0.5552. Test Loss: 0.6981. Train Acc: 0.6471. Test Acc:0.6078
Epoch: 230. Train Loss: 0.5348. Test Loss: 0.6872. Train Acc: 0.6667. Test Acc:0.6078
Epoch: 240. Train Loss: 0.5326. Test Loss: 0.6794. Train Acc: 0.5686. Test Acc:0.6078
Epoch: 250. Train Loss: 0.5122. Test Loss: 0.6948. Train Acc: 0.6863. Test Acc:0.6078
Epoch: 260. Train Loss: 0.5026. Test Loss: 0.683. Train Acc: 0.6471. Test Acc:0.6078
Epoch: 270. Train Loss: 0.5044. Test Loss: 0.6684. Train Acc: 0.6863. Test Acc:0.6275
Epoch: 280. Train Loss: 0.5174. Test Loss: 0.6778. Train Acc: 0.6863. Test Acc:0.6078
Epoch: 290. Train Loss: 0.4718. Test Loss: 0.6647. Train Acc: 0.6863. Test Acc:0.6275
Epoch: 300. Train Loss: 0.4703. Test Loss: 0.6512. Train Acc: 0.6863. Test Acc:0.6275
Epoch: 310. Train Loss: 0.4439. Test Loss: 0.6461. Trai

Epoch: 10. Train Loss: 1.086. Test Loss: 1.084. Train Acc: 0.3137. Test Acc:0.3137
Epoch: 20. Train Loss: 1.085. Test Loss: 1.081. Train Acc: 0.3137. Test Acc:0.3137
Epoch: 30. Train Loss: 1.08. Test Loss: 1.074. Train Acc: 0.3137. Test Acc:0.3137
Epoch: 40. Train Loss: 1.055. Test Loss: 1.046. Train Acc: 0.3137. Test Acc:0.3137
Epoch: 50. Train Loss: 0.9751. Test Loss: 0.9668. Train Acc: 0.4902. Test Acc:0.451
Epoch: 60. Train Loss: 0.8504. Test Loss: 0.8667. Train Acc: 0.5294. Test Acc:0.549
Epoch: 70. Train Loss: 0.7397. Test Loss: 0.7826. Train Acc: 0.5882. Test Acc:0.6078
Epoch: 80. Train Loss: 0.6611. Test Loss: 0.7179. Train Acc: 0.6667. Test Acc:0.6471
Epoch: 90. Train Loss: 0.5818. Test Loss: 0.6626. Train Acc: 0.7255. Test Acc:0.7647
Epoch: 100. Train Loss: 0.5038. Test Loss: 0.6134. Train Acc: 0.7843. Test Acc:0.8235
Epoch: 110. Train Loss: 0.4314. Test Loss: 0.5681. Train Acc: 0.8431. Test Acc:0.8627
Epoch: 120. Train Loss: 0.3869. Test Loss: 0.5261. Train Acc: 0.902. Test 

Epoch: 350. Train Loss: 0.09175. Test Loss: 0.5435. Train Acc: 1.0. Test Acc:0.7059
Epoch: 360. Train Loss: 0.06796. Test Loss: 0.522. Train Acc: 1.0. Test Acc:0.7059
Epoch: 370. Train Loss: 0.05579. Test Loss: 0.4382. Train Acc: 1.0. Test Acc:0.7647
Epoch: 380. Train Loss: 0.04605. Test Loss: 0.4692. Train Acc: 1.0. Test Acc:0.7451
Epoch: 390. Train Loss: 0.06762. Test Loss: 0.5146. Train Acc: 1.0. Test Acc:0.7059
Epoch: 400. Train Loss: 0.0431. Test Loss: 0.5269. Train Acc: 1.0. Test Acc:0.7255
Epoch: 410. Train Loss: 0.03413. Test Loss: 0.3681. Train Acc: 1.0. Test Acc:0.7843
Epoch: 420. Train Loss: 0.03149. Test Loss: 0.378. Train Acc: 1.0. Test Acc:0.8039
Epoch: 430. Train Loss: 0.0271. Test Loss: 0.361. Train Acc: 1.0. Test Acc:0.8235
Epoch: 440. Train Loss: 0.03046. Test Loss: 0.4556. Train Acc: 1.0. Test Acc:0.7255
Epoch: 450. Train Loss: 0.03101. Test Loss: 0.4986. Train Acc: 1.0. Test Acc:0.7255
Epoch: 460. Train Loss: 0.02172. Test Loss: 0.4314. Train Acc: 1.0. Test Acc:0.76

Epoch: 40. Train Loss: 0.9788. Test Loss: 0.9915. Train Acc: 0.3922. Test Acc:0.3922
Epoch: 50. Train Loss: 0.9514. Test Loss: 0.9617. Train Acc: 0.3922. Test Acc:0.4118
Epoch: 60. Train Loss: 0.9138. Test Loss: 0.9252. Train Acc: 0.3922. Test Acc:0.451
Epoch: 70. Train Loss: 0.8769. Test Loss: 0.8843. Train Acc: 0.3922. Test Acc:0.5294
Epoch: 80. Train Loss: 0.8355. Test Loss: 0.8449. Train Acc: 0.451. Test Acc:0.549
Epoch: 90. Train Loss: 0.791. Test Loss: 0.8163. Train Acc: 0.5294. Test Acc:0.549
Epoch: 100. Train Loss: 0.7415. Test Loss: 0.8057. Train Acc: 0.549. Test Acc:0.549
Epoch: 110. Train Loss: 0.6944. Test Loss: 0.7988. Train Acc: 0.5686. Test Acc:0.5882
Epoch: 120. Train Loss: 0.665. Test Loss: 0.8088. Train Acc: 0.5686. Test Acc:0.6078
Epoch: 130. Train Loss: 0.6189. Test Loss: 0.8051. Train Acc: 0.5686. Test Acc:0.6863
Epoch: 140. Train Loss: 0.566. Test Loss: 0.8136. Train Acc: 0.6471. Test Acc:0.6863
Epoch: 150. Train Loss: 0.5322. Test Loss: 0.8354. Train Acc: 0.7451.

...

Epoch: 430. Train Loss: 0.198. Test Loss: 0.3816. Train Acc: 0.9412. Test Acc:0.8824
Epoch: 440. Train Loss: 0.2395. Test Loss: 0.3747. Train Acc: 0.9216. Test Acc:0.8824
Epoch: 450. Train Loss: 0.1659. Test Loss: 0.372. Train Acc: 0.9804. Test Acc:0.8824
Epoch: 460. Train Loss: 0.1916. Test Loss: 0.362. Train Acc: 0.9608. Test Acc:0.902
Epoch: 470. Train Loss: 0.1948. Test Loss: 0.3583. Train Acc: 0.9608. Test Acc:0.902
Epoch: 480. Train Loss: 0.1588. Test Loss: 0.3546. Train Acc: 1.0. Test Acc:0.8824
Epoch: 490. Train Loss: 0.19. Test Loss: 0.3437. Train Acc: 0.902. Test Acc:0.902
Epoch: 500. Train Loss: 0.1762. Test Loss: 0.3315. Train Acc: 0.9608. Test Acc:0.9216
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2996]           4,611           4,611
         Dropout-2       [10, 3, 2996]               0               0
       MaxPool1d-3       [10, 3, 1498] 

Epoch: 120. Train Loss: 0.4635. Test Loss: 0.5769. Train Acc: 0.8235. Test Acc:0.7843
Epoch: 130. Train Loss: 0.4091. Test Loss: 0.5466. Train Acc: 0.8627. Test Acc:0.7843
Epoch: 140. Train Loss: 0.3691. Test Loss: 0.5109. Train Acc: 0.8824. Test Acc:0.8039
Epoch: 150. Train Loss: 0.3186. Test Loss: 0.483. Train Acc: 0.902. Test Acc:0.8431
Epoch: 160. Train Loss: 0.3098. Test Loss: 0.455. Train Acc: 0.8824. Test Acc:0.8824
Epoch: 170. Train Loss: 0.2662. Test Loss: 0.4344. Train Acc: 0.9216. Test Acc:0.8824
Epoch: 180. Train Loss: 0.2483. Test Loss: 0.4131. Train Acc: 0.9216. Test Acc:0.8824
Epoch: 190. Train Loss: 0.2488. Test Loss: 0.3958. Train Acc: 0.902. Test Acc:0.8824
Epoch: 200. Train Loss: 0.2102. Test Loss: 0.3714. Train Acc: 0.9804. Test Acc:0.902
Epoch: 210. Train Loss: 0.1976. Test Loss: 0.3563. Train Acc: 0.9412. Test Acc:0.902
Epoch: 220. Train Loss: 0.1625. Test Loss: 0.3375. Train Acc: 0.9608. Test Acc:0.9412
Epoch: 230. Train Loss: 0.1731. Test Loss: 0.3191. Train Acc

Epoch: 460. Train Loss: 0.01153. Test Loss: 0.1278. Train Acc: 1.0. Test Acc:1.0
Epoch: 470. Train Loss: 0.01247. Test Loss: 0.1181. Train Acc: 1.0. Test Acc:1.0
Epoch: 480. Train Loss: 0.008864. Test Loss: 0.1193. Train Acc: 1.0. Test Acc:1.0
Epoch: 490. Train Loss: 0.01134. Test Loss: 0.1113. Train Acc: 1.0. Test Acc:1.0
Epoch: 500. Train Loss: 0.05592. Test Loss: 0.1299. Train Acc: 0.9804. Test Acc:1.0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2996]           4,611           4,611
         Dropout-2       [10, 3, 2996]               0               0
       MaxPool1d-3       [10, 3, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              30              30
         Dropout-5       [10, 3, 1496]               0               0
       MaxPool1d-6        [10, 3, 748]               0               0
          Conv1d-7    

Epoch: 150. Train Loss: 0.385. Test Loss: 0.5379. Train Acc: 0.8235. Test Acc:0.8039
Epoch: 160. Train Loss: 0.3695. Test Loss: 0.5059. Train Acc: 0.8627. Test Acc:0.8039
Epoch: 170. Train Loss: 0.3274. Test Loss: 0.4841. Train Acc: 0.8824. Test Acc:0.8235
Epoch: 180. Train Loss: 0.3097. Test Loss: 0.4574. Train Acc: 0.902. Test Acc:0.8431
Epoch: 190. Train Loss: 0.2845. Test Loss: 0.4315. Train Acc: 0.902. Test Acc:0.8431
Epoch: 200. Train Loss: 0.2404. Test Loss: 0.4161. Train Acc: 0.9216. Test Acc:0.8431
Epoch: 210. Train Loss: 0.2258. Test Loss: 0.3918. Train Acc: 0.9412. Test Acc:0.8627
Epoch: 220. Train Loss: 0.2114. Test Loss: 0.3788. Train Acc: 0.9608. Test Acc:0.8627
Epoch: 230. Train Loss: 0.2048. Test Loss: 0.3641. Train Acc: 0.9412. Test Acc:0.8824
Epoch: 240. Train Loss: 0.1815. Test Loss: 0.3452. Train Acc: 0.9216. Test Acc:0.8824
Epoch: 250. Train Loss: 0.1683. Test Loss: 0.3293. Train Acc: 0.9412. Test Acc:0.9412
Epoch: 260. Train Loss: 0.1465. Test Loss: 0.3214. Train 

Epoch: 490. Train Loss: 0.05056. Test Loss: 0.1665. Train Acc: 1.0. Test Acc:1.0
Epoch: 500. Train Loss: 0.04475. Test Loss: 0.1635. Train Acc: 1.0. Test Acc:1.0


In [22]:
LangIdConvNet()

LangIdConvNet(
  (conv1): Conv1d(512, 3, kernel_size=(3,), stride=(1,))
  (drop1): Dropout(p=0.5, inplace=False)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
  (drop2): Dropout(p=0.5, inplace=False)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
  (drop3): Dropout(p=0.5, inplace=False)
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)